Add in return flights for round-trips; summarize frequencies by market

In [1]:
import datetime

import pyspark.sql.functions as F
import pyspark.sql.types as T 
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1671213594375_0003,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
BASE_INPUT_DIR = "s3://tvlp-ds-users/kendra-frederick/shopping-grid/agg-raw-data_v4/*"
# date = datetime.date(2022,11,30)
# date_str = date.strftime("%Y%m%d")
# input_dir = "{}/{}".format(BASE_INPUT_DIR, date_str)
input_dir = BASE_INPUT_DIR

df = spark.read.parquet(input_dir)
# df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# df.printSchema()

# df.columns

# df.select(F.min("shop_date"), F.max('shop_date')).show()

# df.groupBy("shop_date").count().orderBy("shop_date").show(40)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# if we want to impose any limits on date range
date_llim = 20221110
date_ulim = 20221213
df_filt = df.filter(F.col("shop_date").between(date_llim, date_ulim)).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# apply fare rank filter here (so it's easier to modify in the future)
df_filt = df_filt.filter(F.col("fare_rank") == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# check
# df2.select(F.min("shop_date"), F.max('shop_date')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

A peek at the data:

In [7]:
df_filt.select('shop_date', 'market', 'out_departure_date', 'out_marketing_cxr', 'out_flight_numbers', 'adt_fare', 'fare_rank',).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+------------------+-----------------+------------------+--------+---------+
|shop_date| market|out_departure_date|out_marketing_cxr|out_flight_numbers|adt_fare|fare_rank|
+---------+-------+------------------+-----------------+------------------+--------+---------+
| 20221205|ATL-EWR|          20221222|             [NK]|              [99]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|             [NK]|              [99]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|             [NK]|              [99]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|             [NK]|              [99]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|             [NK]|            [1173]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|             [NK]|              [99]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|             [NK]|            [1173]|  135.17|        1|
| 20221205|ATL-EWR|          20221222|            

# Analysis by carrier

Notes:
- Crude: list of carriers, no matter if they provide direct lowest-fare flight or not
- Less Crude: distinguish between direct and 1+ stop initineraries

## preprocessing

In [8]:
df_cxr = (df_filt
          .withColumn("out_unique_cxr", F.array_distinct("out_marketing_cxr"))
          .withColumn("out_cxr_all_same", F.when(F.size("out_unique_cxr") == 1, 1).otherwise(0))
          .withColumn("in_unique_cxr", F.array_distinct("in_marketing_cxr"))
          .withColumn("in_cxr_all_same", F.when(F.size("in_unique_cxr") == 1, 1).otherwise(0))
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# # check
# df_cxr.select("market", "in_marketing_cxr", "in_unique_cxr", "in_cxr_all_same").show(10)

# # check

# # df_cxr.filter(F.col("out_num_stops") > 0).select("market", "out_marketing_cxr", "out_unique_cxr", "out_cxr_all_same").show(10)
# df_cxr.filter(F.col("in_num_stops") > 0).select("market", "in_marketing_cxr", "in_unique_cxr", "in_cxr_all_same").show(10)

# # check

# (df_cxr
#  .filter(F.col("in_cxr_all_same") == 0)
#  .select("market", "round_trip", "in_marketing_cxr", "in_unique_cxr", "in_cxr_all_same").show(10)
# )

# # check
# (df_cxr
#  .filter((F.col("in_cxr_all_same") == 0) & (F.col("round_trip") == 1))
#  .select("market", "round_trip", "in_marketing_cxr", "in_unique_cxr", "in_cxr_all_same").show(10)
# )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Note we use `explode_outer` for in-bound carriers so nulls don't get dropped

In [10]:
df_cxr_expl = (df_cxr
               .select("*", F.explode("out_unique_cxr").alias("out_cxr"))
               .select("*", F.explode_outer("in_unique_cxr").alias("in_cxr"))
              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# # check
# df_cxr_expl.count() > df_cxr.count()

# # check
# (df_cxr_expl
# .filter(F.col("round_trip") == 0)
# .select("market", "out_marketing_cxr", "out_unique_cxr", "out_cxr_all_same", "out_cxr")
#  .show()
# )



# # check
# (df_cxr_expl
# .filter((F.col("in_cxr_all_same") == 0) & (F.col("round_trip") == 1))
# .select("market", "in_marketing_cxr", "in_cxr")
#  .show()
# )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### combine out + in

Combine out_unique + in_unique -- all as one string. Can combine with num_stops to determine if a single unique carrier involved multiple legs

Combine in/out unique first, then together 

In [12]:
# resume
df_cxr_expl = (df_cxr_expl
              .withColumn("out_uni_cxr_concat", F.concat_ws("-", "out_unique_cxr"))
              .withColumn("in_uni_cxr_concat", F.concat_ws("-", "in_unique_cxr"))
              .withColumn("rt_cxrs", F.concat_ws("_", "out_uni_cxr_concat", "in_uni_cxr_concat"))
              .withColumn("out_in_same_cxr", F.when(F.col("out_cxr") == F.col("in_cxr"), 1).otherwise(0))
              )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# (df_cxr_expl
#  # .filter(F.col("out_cxr_all_same") == 0)
#  .filter(F.col("round_trip") == 0)
#  .select('out_marketing_cxr', 'out_unique_cxr', 'out_uni_cxr_concat', 'in_marketing_cxr', 'rt_cxrs')
#  .show() 
# )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## "crude" (single carrier)
regardless of num stops / inter-line

In [14]:
out_cxr_crude = (
    df_cxr_expl
    .filter(F.col("fare_rank") == 1)
    .groupBy("market", "round_trip", "out_cxr")
    .agg(F.count("*").alias("count"))
)
# alt to additional processing below: add round_trip == 1 filter here
rt_in_cxr_crude = (
    df_cxr_expl
    .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
    .groupBy("market", "round_trip", "in_cxr")
    .agg(F.count("*").alias("count"))
)

# combine out_rt + in counts
same_cxr_crude = (
    df_cxr_expl
    .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
    .groupBy("market", "round_trip", "out_cxr",  "out_in_same_cxr") # don't need in_cxr 
    .agg(F.count("*").alias("count"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# # check
# out_cxr_m_grp_crude.show(10)

# # check
# out_cxr_m_grp_crude.groupBy("round_trip").count().show()

# # check
# in_cxr_m_grp_crude.show(10)

# in_cxr_m_grp_crude.groupBy("round_trip").count().show()

# in_cxr_m_grp_crude.filter(F.col("round_trip") == 0).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# process
# in_cxr_m_grp_crude = in_cxr_m_grp_crude.dropna(subset=["in_cxr"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# check
# in_cxr_m_grp_crude.filter(F.col("round_trip") == 0).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## "nuanced" (inter-line)
exclude num stops for now

In [18]:
out_cxr_nuanced = (
    df_cxr_expl
    .filter(F.col("fare_rank") == 1) 
    .groupBy("market", "round_trip", "out_uni_cxr_concat") # "out_num_stops",
    .agg(F.count("*").alias("count"))
)
rt_in_cxr_nuanced = (
    df_cxr_expl
    .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
    # not necessary to include round-trip here, but makes processing below simpler
    .groupBy("market", "round_trip", "in_uni_cxr_concat") # "in_num_stops",
    .agg(F.count("*").alias("count"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# # check
# out_cxr_m_grp_nuanced.show(10)

# # check
# in_cxr_m_grp_nuanced.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## window func to calc pct by market

In [20]:
# just use a single window func to make it easy
w1 = Window.partitionBy("market", "round_trip")
# w1b = Window.partitionBy("market")


def apply_window_1(df, w):
    df = (df
          .withColumn("count_market", F.sum("count").over(w))
          .withColumn("pct_count", F.col("count") / F.col("count_market"))
         )
    return df

w2 = (Window
      .partitionBy("market", "round_trip")
      .orderBy(F.desc("pct_count"))
      .rowsBetween(Window.unboundedPreceding, Window.currentRow)
     )
# w2b = (Window
#       .partitionBy("market")
#       .orderBy(F.desc("pct_count"))
#       .rowsBetween(Window.unboundedPreceding, Window.currentRow)
#      )

def apply_window_2(df, w2):
    df = df.withColumn("cum_pct", F.sum("pct_count").over(w2))
    return df

In [20]:
out_cxr_crude = apply_window_1(out_cxr_crude, w1)
out_cxr_crude = apply_window_2(out_cxr_crude, w2)

rt_in_cxr_crude = apply_window_1(rt_in_cxr_crude, w1)
rt_in_cxr_crude = apply_window_2(rt_in_cxr_crude, w2)

out_cxr_nuanced = apply_window_1(out_cxr_nuanced, w1)
out_cxr_nuanced = apply_window_2(out_cxr_nuanced, w2)

rt_in_cxr_nuanced = apply_window_1(rt_in_cxr_nuanced, w1)
rt_in_cxr_nuanced = apply_window_2(rt_in_cxr_nuanced, w2)

same_cxr_crude = apply_window_1(same_cxr_crude, w1)
same_cxr_crude = apply_window_2(same_cxr_crude, w2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# split out into ow & rt
ow_out_cxr_crude = out_cxr_crude.filter(F.col("round_trip") == 0)
rt_out_cxr_crude = out_cxr_crude.filter(F.col("round_trip") == 1)

ow_out_cxr_nuanced = out_cxr_nuanced.filter(F.col("round_trip") == 0)
rt_out_cxr_nuanced = out_cxr_nuanced.filter(F.col("round_trip") == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# # check
# same_cxr_m_grp_crude.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Analysis by flight

## preprocess

### Concat carrier + flight no

In [23]:
# fi = flight info = carrier + flight no
df_fi = (df_filt
       .withColumn("out_flight_nos_str", F.col("out_flight_numbers").cast('array<string>'))
       .withColumn("out_cxr_fn_zip", F.arrays_zip(F.col("out_marketing_cxr"), F.col("out_flight_nos_str")))
       .withColumn('out_zip_concat', F.expr("transform(out_cxr_fn_zip, x -> concat_ws('-', x.out_marketing_cxr, x.out_flight_nos_str))"))
       .withColumn("in_flight_nos_str", F.col("in_flight_numbers").cast('array<string>'))
       .withColumn("in_cxr_fn_zip", F.arrays_zip(F.col("in_marketing_cxr"), F.col("in_flight_nos_str")))
       .withColumn('in_zip_concat', F.expr("transform(in_cxr_fn_zip, x -> concat_ws('-', x.in_marketing_cxr, x.in_flight_nos_str))"))
      )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# # check
# df_combined.groupBy("round_trip").count().show()

# # df_combined.select("out_marketing_cxr",  "out_flight_nos_str", "out_cxr_fn_zip", "out_zip_concat").show(10) #.printSchema()
# df_combined.select("in_marketing_cxr",  "in_flight_nos_str", "in_cxr_fn_zip", "in_zip_concat").show(10)

# df2.columns

# (df_combined
#  .filter((F.col("round_trip") == 0)) #& (F.col("in_num_stops") > 0))
#  .select("out_marketing_cxr",  "out_flight_nos_str", "out_cxr_fn_zip", "out_zip_concat")
#  .show(10)
# )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

From here we can explode array elements to rows, or combine multi-legs into one entity

### combine multi-legs into one string

In [25]:
df_fi_one_str = (df_fi
              .withColumn("out_cxr_fn_single", F.concat_ws(".", "out_zip_concat"))
              .withColumn("in_cxr_fn_single", F.concat_ws(".", "in_zip_concat"))
              .withColumn("rt_cxr_fn_single", F.concat_ws("_", "out_cxr_fn_single", "in_cxr_fn_single"))
             )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# df_one_str.select("out_cxr_fn_single", "in_cxr_fn_single", "rt_cxr_fn_single").show(10)

# df_one_str.filter(F.col("round_trip") == 0).select("out_cxr_fn_single", "in_cxr_fn_single", "rt_cxr_fn_single").show(10)

# df_one_str.filter(F.col("out_num_stops") > 0).select("out_cxr_fn_single", "in_cxr_fn_single", "rt_cxr_fn_single").show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### each leg separately (explode cols --> rows)

This explodes cols --> rows to count by leg. Note it doesn't make sense to concat out + in here (since we want to analyze by individual leg -- keep them separate)

In [27]:
df_fi_expl = (df_fi
           .select("*", F.posexplode("out_zip_concat").alias("out_leg_num", "out_leg_fn"))
           .withColumn("out_leg_num", F.col("out_leg_num") + 1)
           .select("*", F.posexplode_outer("in_zip_concat").alias("in_leg_num", "in_leg_fn"))
           .withColumn("in_leg_num", F.col("in_leg_num") + 1)
          )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# df_fi_expl.groupBy("round_trip").count().show()

# df_fi_expl.select("out_zip_concat", "out_leg_num", "out_leg_fn", "out_num_stops").show(10)

# df_fi_expl.select("in_zip_concat", "in_leg_num", "in_leg_fn", "in_num_stops").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## groupBy market & count

We separate into out-bound and in-bound legs. Note we'll later want to divide the out-bound df into one-way and round-trip

Note: for the in-bound (round-trip), it's not nec to include round_trip in the groupBy, but it makes processing below simpler

In [29]:
out_fi_per_leg = (df_fi_expl
                    .filter(F.col("fare_rank") == 1)
                    .groupBy("market", "round_trip", "out_leg_fn")
                    .agg(F.count("*").alias("count"))
                 )

rt_in_fi_per_leg = (df_fi_expl
                    .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
                    .groupBy("market", "round_trip", "in_leg_fn")
                    .agg(F.count("*").alias("count"))
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# ow_out_fi_one_str = (df_one_str
#                       .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 0))
#                       .groupBy("market", "out_cxr_fn_single")
#                       .agg(F.count("*").alias("count"))
#                      )

# rt_out_fi_one_str = (df_one_str
#                       .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
#                       .groupBy("market", "out_cxr_fn_single")
#                       .agg(F.count("*").alias("count"))
#                      )

In [71]:
out_fi_one_str = (df_fi_one_str
                      .filter(F.col("fare_rank") == 1)
                      .groupBy("market", "round_trip", "out_cxr_fn_single")
                      .agg(F.count("*").alias("count"))
                     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
rt_in_fi_one_str = (df_fi_one_str
                      .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
                      .groupBy("market", "round_trip", "in_cxr_fn_single")
                      .agg(F.count("*").alias("count"))
                     )

rt_fi_one_str = (df_fi_one_str
                      .filter((F.col("fare_rank") == 1) & (F.col("round_trip") == 1))
                      .groupBy("market", "round_trip", "rt_cxr_fn_single")
                      .agg(F.count("*").alias("count"))
                     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## window func to calc pct by market

Note: these are the same window functions we used above; we could combine definitions

In [38]:
out_fi_per_leg = apply_window_1(out_fi_per_leg, w1)
out_fi_per_leg = apply_window_2(out_fi_per_leg, w2)

rt_in_fi_per_leg = apply_window_1(rt_in_fi_per_leg, w1)
rt_in_fi_per_leg = apply_window_2(rt_in_fi_per_leg, w2)

In [72]:
out_fi_one_str = apply_window_1(out_fi_one_str, w1)
out_fi_one_str = apply_window_2(out_fi_one_str, w2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
rt_in_fi_one_str = apply_window_1(rt_in_fi_one_str, w1)
rt_in_fi_one_str = apply_window_2(rt_in_fi_one_str, w2)

rt_fi_one_str = apply_window_1(rt_fi_one_str, w1)
rt_fi_one_str = apply_window_2(rt_fi_one_str, w2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# out_fi_per_leg_grp.groupBy("round_trip").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
# split into ow & rt
ow_out_fi_per_leg = out_fi_per_leg.filter(F.col("round_trip") == 0)
rt_out_fi_per_leg = out_fi_per_leg.filter(F.col("round_trip") == 1)

In [73]:
ow_out_fi_one_str = out_fi_one_str.filter(F.col("round_trip") == 0)
rt_out_fi_one_str = out_fi_one_str.filter(F.col("round_trip") == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# out_ow_fi_per_leg_grp.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Overall results

In [35]:
# out_ow_fi_per_leg.orderBy(F.desc("pct_count")).show()

# out_rt_fi_per_leg.orderBy(F.desc("pct_count")).show(truncate=False)

# out_ow_fi_per_leg.orderBy(F.desc("pct_count")).show(truncate=False)

# out_rt_fi_per_leg.orderBy(F.desc("pct_count")).show(truncate=False)

# rt_fi_one_str.orderBy(F.desc("pct_count")).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Results by market

In [44]:
def show_market_analysis(market):
    # mdf = cxr_market_grp_crude.filter(F.col("market") == market)
    print("Carriers - any leg - one-way")
    (ow_out_cxr_crude.filter(F.col("market") == market)
     .drop("count_market").orderBy(F.desc("pct_count")).show()
    )
    
    print("Carriers - any leg - Round-trip")
    same_cxr_crude.filter(F.col("market") == market).drop("count_market").orderBy(F.desc("pct_count")).show()
    
    print("Interline carriers - one-way (out-bound)")
    (ow_out_cxr_nuanced
     .filter(F.col("market") == market)
     .drop("count_market")
     .orderBy(F.desc("pct_count"))
     .show()
    )

    print("Interline carriers - round-trip, out-bound")
    (rt_out_cxr_nuanced
     .filter(F.col("market") == market)
     .drop("count_market")
     .orderBy(F.desc("pct_count"))
     .show()
    )

    print("Interline carriers - round-trip, in-bound")
    (rt_in_cxr_nuanced
     .filter(F.col("market") == market)
     .drop("count_market")
     .orderBy(F.desc("pct_count"))
     .show()
    )
    
    
    print("Individual Legs - one-way (out-bound)")
    mdf_leg = ow_out_fi_per_leg.filter(F.col("market") == market)
    mdf_leg.orderBy(F.desc("count")).drop("count_market").show()
    
    print("Individual Legs - round-trip, out-bound")
    mdf_leg = rt_out_fi_per_leg.filter(F.col("market") == market)
    mdf_leg.orderBy(F.desc("count")).drop("count_market").show()
    
    print("Individual Legs - round-trip, in-bound")
    mdf_leg = rt_in_fi_per_leg.filter(F.col("market") == market)
    mdf_leg.orderBy(F.desc("count")).drop("count_market").show()    

    
    print("Itineraries - one-way (out-bound)")
    mdf_single = ow_out_fi_one_str.filter(F.col("market") == market)
    mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show()

    print("Itineraries - round-trip, out-bound")
    mdf_single = rt_out_fi_one_str.filter(F.col("market") == market)
    mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show(truncate=False)
    
    print("Itineraries - round-trip, in-bound")
    mdf_single = rt_in_fi_one_str.filter(F.col("market") == market)
    mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show(truncate=False)
    
    print("Itineraries - round-trip, out+in")
    mdf_single = rt_fi_one_str.filter(F.col("market") == market)
    mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show(truncate=False)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## JFK-LHR

In [74]:
show_market_analysis("JFK-LHR")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carriers - any leg - one-way
+-------+----------+-------+------+--------------------+------------------+
| market|round_trip|out_cxr| count|           pct_count|           cum_pct|
+-------+----------+-------+------+--------------------+------------------+
|JFK-LHR|         0|     TP|866038|  0.7634239029348218|0.7634239029348218|
|JFK-LHR|         0|     DL|114666| 0.10107958917960214|0.8645034921144239|
|JFK-LHR|         0|     VS|112383| 0.09906709461192705| 0.963570586726351|
|JFK-LHR|         0|     WS| 19658| 0.01732878590072575|0.9808993726270767|
|JFK-LHR|         0|     FI| 14246|0.012558036623346171|0.9934574092504229|
|JFK-LHR|         0|     TK|  1620|0.001428051335800...|0.9948854605862238|
|JFK-LHR|         0|     AZ|  1224|0.001078972120382...|0.9959644327066068|
|JFK-LHR|         0|     AF|  1130|9.961098823796978E-4|0.9969605425889866|
|JFK-LHR|         0|     JU|   682|6.011919821088087E-4|0.9975617345710954|
|JFK-LHR|         0|     SK|   577|5.086330992328191E-4|0.9

## LHR-JFK

In [ ]:
show_market_analysis("LHR-JFK")

## ATL-EWR

In [75]:
show_market_analysis("ATL-EWR")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carriers - any leg - one-way
+-------+----------+-------+------+--------------------+------------------+
| market|round_trip|out_cxr| count|           pct_count|           cum_pct|
+-------+----------+-------+------+--------------------+------------------+
|ATL-EWR|         0|     NK|210032|  0.6462065761499955|0.6462065761499955|
|ATL-EWR|         0|     UA| 75605| 0.23261430729517604|0.8788208834451716|
|ATL-EWR|         0|     DL| 36129| 0.11115828725967085|0.9899791707048424|
|ATL-EWR|         0|     AA|  3139|0.009657778064936944|0.9996369487697794|
|ATL-EWR|         0|     B6|   113|3.476677035163665...|0.9999846164732957|
|ATL-EWR|         0|     AS|     5|1.538352670426400...|               1.0|
+-------+----------+-------+------+--------------------+------------------+

Carriers - any leg - Round-trip
+-------+----------+-------+---------------+------+--------------------+------------------+
| market|round_trip|out_cxr|out_in_same_cxr| count|           pct_count|           cum

## SFO-LAX

In [86]:
market = 'SFO-LAX'
show_market_analysis(market)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carriers - any leg - one-way
+-------+----------+-------+------+--------------------+-------------------+
| market|round_trip|out_cxr| count|           pct_count|            cum_pct|
+-------+----------+-------+------+--------------------+-------------------+
|SFO-LAX|         0|     UA|874146| 0.36748989151189687|0.36748989151189687|
|SFO-LAX|         0|     AS|720695| 0.30297928190847584| 0.6704691734203727|
|SFO-LAX|         0|     DL|540880| 0.22738527948529738| 0.8978544529056701|
|SFO-LAX|         0|     B6|141188|0.059355259650884055| 0.9572097125565542|
|SFO-LAX|         0|     WN| 98185| 0.04127685191958276| 0.9984865644761369|
|SFO-LAX|         0|     AA|  3600|0.001513435523863...|                1.0|
+-------+----------+-------+------+--------------------+-------------------+

Carriers - any leg - Round-trip
+-------+----------+-------+---------------+-------+--------------------+-------------------+
| market|round_trip|out_cxr|out_in_same_cxr|  count|           pct_count| 

## others

In [ ]:
market = 'LGA-MIA'
show_market_analysis(market)

In [ ]:
market = 'OAK-LAS'
show_market_analysis(market)

In [ ]:
market = 'LAX-SFO'
show_market_analysis(market)

In [ ]:
market = 'LAX-JFK'
show_market_analysis(market)

In [ ]:
market = 'JFK-LAX'
show_market_analysis(market)

In [ ]:
market = 'EWR-CDG'
show_market_analysis(market)

In [ ]:
market_one_str_grp.select("market").distinct().show()

# Higher level summary

Number of carriers serving market

In [51]:
df_summ_1 = ow_out_cxr_crude.groupBy("market").count().orderBy("market").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+
| market|count|
+-------+-----+
|ATL-EWR|    6|
|EWR-CDG|   21|
|EWR-LHR|   20|
|JFK-LAX|    5|
|JFK-LHR|   21|
|LAX-EWR|    7|
|LAX-JFK|    6|
|LAX-SFO|    6|
|LGA-MIA|    7|
|LHR-EWR|    8|
|LHR-JFK|   14|
|LHR-LAX|   21|
|OAK-LAS|    6|
|SFO-LAX|    6|
+-------+-----+

Number of carriers it takes for a given market to reach 99% cume 

In [55]:
cutoff = 0.99

w3 = Window.partitionBy("market").orderBy('cum_pct')

result_df = ow_out_cxr_crude.withColumn("row_num", F.row_number().over(w3))
filt_result_df = result_df.filter(F.col("cum_pct") <= cutoff)
final_result = filt_result_df.groupBy("market").agg(F.max("row_num"))

In [55]:
final_result.orderBy("market").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|max(row_num)|
+-------+------------+
|ATL-EWR|           3|
|EWR-CDG|           9|
|EWR-LHR|           5|
|JFK-LAX|           3|
|JFK-LHR|           4|
|LAX-EWR|           5|
|LAX-JFK|           3|
|LAX-SFO|           3|
|LGA-MIA|           3|
|LHR-EWR|           2|
|LHR-JFK|           5|
|LHR-LAX|           8|
|OAK-LAS|           2|
|SFO-LAX|           4|
+-------+------------+

## num to get to X %

In [58]:
def count_num_records_to_cutoff(df, cutoff, things="things"):
    w3 = Window.partitionBy("market").orderBy('cum_pct')

    window_df = df.withColumn("row_num", F.row_number().over(w3))
    filt_df = window_df.filter(F.col("cum_pct") <= cutoff)
    # could add a collect_list agg here 
    final_result = filt_df.groupBy("market").agg(F.max("row_num").alias(f"num_{things}"))
    final_result.orderBy("market").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
count_num_records_to_cutoff(ow_out_cxr_crude, 0.99, "carriers")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|           3|
|EWR-CDG|           9|
|EWR-LHR|           5|
|JFK-LAX|           3|
|JFK-LHR|           4|
|LAX-EWR|           5|
|LAX-JFK|           3|
|LAX-SFO|           3|
|LGA-MIA|           3|
|LHR-EWR|           2|
|LHR-JFK|           5|
|LHR-LAX|           8|
|OAK-LAS|           2|
|SFO-LAX|           4|
+-------+------------+

Num carriers, round-trip, including inter-line

In [60]:
count_num_records_to_cutoff(rt_out_cxr_nuanced, 0.99, "carriers")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|           2|
|EWR-CDG|          16|
|EWR-LHR|          16|
|JFK-LAX|           2|
|JFK-LHR|          19|
|LAX-EWR|           4|
|LAX-JFK|           2|
|LAX-SFO|           4|
|LGA-MIA|           3|
|LHR-EWR|          12|
|LHR-JFK|          13|
|LHR-LAX|          23|
|OAK-LAS|           4|
|SFO-LAX|           4|
+-------+------------+

Num legs - one-way

In [61]:
count_num_records_to_cutoff(ow_out_fi_per_leg, 0.99, "legs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|          60|
|EWR-CDG|          83|
|EWR-LHR|          34|
|JFK-LAX|         119|
|JFK-LHR|          36|
|LAX-EWR|         205|
|LAX-JFK|         157|
|LAX-SFO|          66|
|LGA-MIA|          26|
|LHR-EWR|          13|
|LHR-JFK|          24|
|LHR-LAX|          68|
|OAK-LAS|          32|
|SFO-LAX|          79|
+-------+------------+

Num legs - round-trip, out-bound

In [62]:
count_num_records_to_cutoff(rt_out_fi_per_leg, 0.99, "legs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|          92|
|EWR-CDG|         152|
|EWR-LHR|         206|
|JFK-LAX|         164|
|JFK-LHR|         301|
|LAX-EWR|         218|
|LAX-JFK|         180|
|LAX-SFO|          81|
|LGA-MIA|          43|
|LHR-EWR|         155|
|LHR-JFK|         244|
|LHR-LAX|         457|
|OAK-LAS|          65|
|SFO-LAX|          79|
+-------+------------+

Num legs - round-trip, in-bound

In [76]:
count_num_records_to_cutoff(rt_in_fi_per_leg, 0.99, "legs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_legs|
+-------+--------+
|ATL-EWR|     101|
|EWR-CDG|     122|
|EWR-LHR|     219|
|JFK-LAX|     187|
|JFK-LHR|     297|
|LAX-EWR|     208|
|LAX-JFK|     146|
|LAX-SFO|      88|
|LGA-MIA|      52|
|LHR-EWR|     160|
|LHR-JFK|     171|
|LHR-LAX|     427|
|OAK-LAS|      66|
|SFO-LAX|      78|
+-------+--------+

Num itin - one-way

In [77]:
count_num_records_to_cutoff(ow_out_fi_one_str, 0.99, "itin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_itin|
+-------+--------+
|ATL-EWR|      49|
|EWR-CDG|      81|
|EWR-LHR|      50|
|JFK-LAX|      86|
|JFK-LHR|      36|
|LAX-EWR|     146|
|LAX-JFK|     118|
|LAX-SFO|      66|
|LGA-MIA|      24|
|LHR-EWR|      21|
|LHR-JFK|      24|
|LHR-LAX|     108|
|OAK-LAS|      18|
|SFO-LAX|      78|
+-------+--------+

Num itin - round-trip, out-bound

In [69]:
count_num_records_to_cutoff(rt_out_fi_per_leg, 0.99, "itin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|          92|
|EWR-CDG|         152|
|EWR-LHR|         206|
|JFK-LAX|         164|
|JFK-LHR|         301|
|LAX-EWR|         218|
|LAX-JFK|         180|
|LAX-SFO|          81|
|LGA-MIA|          43|
|LHR-EWR|         155|
|LHR-JFK|         244|
|LHR-LAX|         457|
|OAK-LAS|          65|
|SFO-LAX|          79|
+-------+------------+

Num itin - round-trip, out+in

In [70]:
count_num_records_to_cutoff(rt_fi_one_str, 0.99, "itin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_itin|
+-------+--------+
|ATL-EWR|    1381|
|EWR-CDG|    8076|
|EWR-LHR|   10985|
|JFK-LAX|    3098|
|JFK-LHR|   11584|
|LAX-EWR|    7805|
|LAX-JFK|    2502|
|LAX-SFO|    1341|
|LGA-MIA|     435|
|LHR-EWR|    5957|
|LHR-JFK|    6059|
|LHR-LAX|   21921|
|OAK-LAS|     684|
|SFO-LAX|    1307|
+-------+--------+

## reverse: top X = Y %

In [82]:
def top_num_records_to_cutoff(df, top_n):
    w3 = Window.partitionBy("market").orderBy('cum_pct')
    window_df = df.withColumn("row_num", F.row_number().over(w3))
    
    filt_df = window_df.filter(F.col("row_num") <= top_n)
    # could add a collect_list agg here 
    final_result = filt_df.groupBy("market").agg(F.max("cum_pct").alias("pct_lowest_fares"))
    final_result.orderBy("market").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Num itin - one-way

In [81]:
top_num_records_to_cutoff(ow_out_fi_one_str, 25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| market|  pct_lowest_fares|
+-------+------------------+
|ATL-EWR|0.9408103426526736|
|EWR-CDG|0.8823952062712731|
|EWR-LHR|0.9707849969663718|
|JFK-LAX|0.8654412839589761|
|JFK-LHR|0.9659742781693096|
|LAX-EWR|0.9126270602575656|
|LAX-JFK| 0.827081804080738|
|LAX-SFO|0.7888481520339221|
|LGA-MIA|0.9906621991697987|
|LHR-EWR|0.9958209445633411|
|LHR-JFK|0.9904760561279908|
|LHR-LAX|0.8808668119577285|
|OAK-LAS|0.9944319530624258|
|SFO-LAX|0.7426255752105986|
+-------+------------------+

Num itin - one-way

In [85]:
top_num_records_to_cutoff(ow_out_fi_per_leg, 25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| market|  pct_lowest_fares|
+-------+------------------+
|ATL-EWR|0.9196492047154695|
|EWR-CDG| 0.873729138446597|
|EWR-LHR|0.9818448833998286|
|JFK-LAX|0.8307870703758276|
|JFK-LHR|0.9689295640185027|
|LAX-EWR|0.8594295756308284|
|LAX-JFK|0.7793942943066591|
|LAX-SFO|0.7887177815676942|
|LGA-MIA|0.9857631466438648|
|LHR-EWR|0.9998261771364992|
|LHR-JFK|0.9907304292840298|
|LHR-LAX|   0.8798940465187|
|OAK-LAS|0.9826090884452074|
|SFO-LAX|0.7422735818056981|
+-------+------------------+

Carriers

In [84]:
top_num_records_to_cutoff(ow_out_cxr_crude, 8)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| market|  pct_lowest_fares|
+-------+------------------+
|ATL-EWR|               1.0|
|EWR-CDG|0.9834133030130756|
|EWR-LHR|0.9995538280693627|
|JFK-LAX|               1.0|
|JFK-LHR|0.9969605425889866|
|LAX-EWR|               1.0|
|LAX-JFK|               1.0|
|LAX-SFO|1.0000000000000002|
|LGA-MIA|               1.0|
|LHR-EWR|0.9999999999999999|
|LHR-JFK|0.9991939684281197|
|LHR-LAX|0.9819964635910625|
|OAK-LAS|0.9999999999999999|
|SFO-LAX|               1.0|
+-------+------------------+

In [ ]:
# print("Carriers - any leg - one-way")
# (ow_out_cxr_crude.filter(F.col("market") == market)
#  .drop("count_market").orderBy(F.desc("pct_count")).show()
# )

# print("Carriers - any leg - Round-trip")
# same_cxr_crude.filter(F.col("market") == market).drop("count_market").orderBy(F.desc("pct_count")).show()

# print("Interline carriers - one-way (out-bound)")
# (ow_out_cxr_nuanced
#  .filter(F.col("market") == market)
#  .drop("count_market")
#  .orderBy(F.desc("pct_count"))
#  .show()
# )

# print("Interline carriers - round-trip, out-bound")
# (rt_out_cxr_nuanced
#  .filter(F.col("market") == market)
#  .drop("count_market")
#  .orderBy(F.desc("pct_count"))
#  .show()
# )

# print("Interline carriers - round-trip, in-bound")
# (rt_in_cxr_nuanced
#  .filter(F.col("market") == market)
#  .drop("count_market")
#  .orderBy(F.desc("pct_count"))
#  .show()
# )


# print("Individual Legs - one-way (out-bound)")
# mdf_leg = ow_out_fi_per_leg.filter(F.col("market") == market)
# mdf_leg.orderBy(F.desc("count")).drop("count_market").show()

# print("Individual Legs - round-trip, out-bound")
# mdf_leg = rt_out_fi_per_leg.filter(F.col("market") == market)
# mdf_leg.orderBy(F.desc("count")).drop("count_market").show()

# print("Individual Legs - round-trip, in-bound")
# mdf_leg = rt_in_fi_per_leg.filter(F.col("market") == market)
# mdf_leg.orderBy(F.desc("count")).drop("count_market").show()    


# print("Itineraries - one-way (out-bound)")
# mdf_single = ow_out_fi_one_str.filter(F.col("market") == market)
# mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show()

# print("Itineraries - round-trip, out-bound")
# mdf_single = rt_out_fi_one_str.filter(F.col("market") == market)
# mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show(truncate=False)

# print("Itineraries - round-trip, in-bound")
# mdf_single = rt_in_fi_one_str.filter(F.col("market") == market)
# mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show(truncate=False)

# print("Itineraries - round-trip, out+in")
# mdf_single = rt_fi_one_str.filter(F.col("market") == market)
# mdf_single.orderBy(F.desc("pct_count")).drop("count_market").show(truncate=False)

# NEXT STEPS

- repeat, lifting filter on fare_rank = 1 (i.e. analyze for 3 lowest fares)
- also break out by
    - DTD
    - DOW